In [1]:
import librosa
import pandas as pd
import numpy as np
import random
import shutil
import os

### 메타데이터 확인

In [5]:
file_path = "/home/woongjae/noise-tracing/Datasets/LibriSpeech/meta.csv"
meta_data = pd.read_csv(file_path, sep=',')
print(meta_data.head())

                                         path Utterence name (file name)  \
0  Real/LibriSpeech/wav16/7241-90850-0013.wav        7241-90850-0013.wav   
1  Real/LibriSpeech/wav16/7241-90852-0056.wav        7241-90852-0056.wav   
2  Real/LibriSpeech/wav16/7241-90852-0014.wav        7241-90852-0014.wav   
3  Real/LibriSpeech/wav16/7241-90850-0034.wav        7241-90850-0034.wav   
4  Real/LibriSpeech/wav16/7241-90852-0032.wav        7241-90852-0032.wav   

  TTS or VC Is multi-speaker? Language Noise type 1 Source link  \
0         -                No  English            -           -   
1         -                No  English            -           -   
2         -                No  English            -           -   
3         -                No  English            -           -   
4         -                No  English            -           -   

               utt        group Model  Speaker name Gender    Age     label  
0  7241-90850-0013  LibriSpeech     -  AdrianBisson   Male  Ad

In [9]:
print(meta_data['Noise type 1'].unique())
speaker_counts = meta_data['Speaker name'].value_counts()
print(speaker_counts)

['-']
AdrianBisson         100
Mike Rosenlof        100
Mitchell Dwyer       100
Monique              100
Mur Lafferty         100
                    ... 
James Smith          100
Jan Dawn Doronila    100
Janet                100
Jason Isbell         100
wvthcomp             100
Name: Speaker name, Length: 166, dtype: int64


### 각 화자별로 100개씩 랜덤으로 뽑기

In [2]:
meta_path = "/home/woongjae/noise-tracing/Datasets/LibriSpeech/meta.csv"
output_dir = "/home/woongjae/noise-tracing/new_dataset/Dataset/Bonafide/Clean"
base_dir = '/home/woongjae/noise-tracing/Datasets'  # 오디오 파일 기준 루트
os.makedirs(output_dir, exist_ok=True)

# 메타데이터 불러오기
df = pd.read_csv(meta_path)

# 각 화자당 100개 이상 샘플이 있는 화자만 선택
speaker_counts = df['Speaker name'].value_counts()
eligible_speakers = speaker_counts[speaker_counts >= 100].index.tolist()

# 메타데이터 저장용 리스트
new_meta = []

# 각 화자에서 100개씩 랜덤 추출 + 복사 + 메타정보 수집
for speaker in eligible_speakers:
    speaker_df = df[df['Speaker name'] == speaker]
    sampled_df = speaker_df.sample(n=100, random_state=42)

    for _, row in sampled_df.iterrows():
        rel_path = row['path'].replace('Real/', '')
        abs_path = os.path.join(base_dir, rel_path)
        dst = os.path.join(output_dir, os.path.basename(rel_path))

        # 파일 복사
        shutil.copy(abs_path, dst)

        # 메타데이터 항목 저장
        new_meta.append({
            'File_path': dst,
            'utt': row['utt'],
            'speaker ID': row['Speaker name'],
            'gender': row['Gender'],
            'Group': 'LibriSpeech',
            'Label': 'bonafide'
        })

# 메타데이터 저장
meta_df = pd.DataFrame(new_meta)
meta_df.to_csv('meta_new_LibriSpeech.csv', index=False)